In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer


In [2]:
accidents = pd.read_csv("Accident_Information.csv", encoding = "Latin1")
vehicles = pd.read_csv("Vehicle_Information.csv", encoding = "Latin1")

/var/folders/gn/clhq9k9140jfd9jjcjjjpqvh0000gn/T/ipykernel_27041/2399508801.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents = pd.read_csv("Accident_Information.csv", encoding = "Latin1")


In [3]:
train_raw = pd.merge(accidents,vehicles, on = "Accident_Index", how = "inner")
train_raw

,Accident_Index,1st_Road_Class,1st_Road_Number,2nd_Road_Class,2nd_Road_Number,Accident_Severity,Carriageway_Hazards,Date,Day_of_Week,Did_Police_Officer_Attend_Scene_of_Accident,...,Skidding_and_Overturning,Towing_and_Articulation,Vehicle_Leaving_Carriageway,Vehicle_Location.Restricted_Lane,Vehicle_Manoeuvre,Vehicle_Reference,Vehicle_Type,Was_Vehicle_Left_Hand_Drive,X1st_Point_of_Impact,Year_y
0,200501BS00002,B,450.0,C,0.0,Slight,NaN,2005-01-05,Wednesday,1.0,...,NaN,No tow/articulation,Did not leave carriageway,0.0,Slowing or stopping,1,Bus or coach (17 or more pass seats),No,Nearside,2005
1,200501BS00003,C,0.0,NaN,0.0,Slight,NaN,2005-01-06,Thursday,1.0,...,NaN,No tow/articulation,Did not leave carriageway,0.0,Going ahead right-hand bend,1,Bus or coach (17 or more pass seats),No,Nearside,2005
2,200501BS00004,A,3220.0,NaN,0.0,Slight,NaN,2005-01-07,Friday,1.0,...,NaN,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Car,No,Front,2005
3,200501BS00005,Unclassified,0.0,NaN,0.0,Slight,NaN,2005-01-10,Monday,1.0,...,Skidded,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Motorcycle 125cc and under,No,Front,2005
4,200501BS00006,Unclassified,0.0,NaN,0.0,Slight,NaN,2005-01-11,Tuesday,1.0,...,NaN,No tow/articulation,Did not leave carriageway,0.0,Moving off,1,Car,No,Did not impact,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058403,2016984131116,B,7020.0,C,49.0,Slight,NaN,2016-11-01,Tuesday,1.0,...,NaN,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Car,No,Front,2016
2058404,2016984131116,B,7020.0,C,49.0,Slight,NaN,2016-11-01,Tuesday,1.0,...,NaN,No tow/articulation,Offside,0.0,Going ahead other,2,Car,No,Front,2016
2058405,2016984131216,A(M),74.0,NaN,0.0,Slight,NaN,2016-10-27,Thursday,1.0,...,NaN,No tow/articulation,Offside on to central reservation,0.0,Going ahead other,1,Goods 7.5 tonnes mgw and over,No,Offside,2016
2058406,2016984131316,B,724.0,NaN,0.0,Slight,NaN,2016-10-29,Saturday,1.0,...,Skidded,No tow/articulation,Offside,0.0,Going ahead other,1,Car,No,Front,2016


In [4]:
train_raw.isnull().sum().sort_values( ascending = False)

Carriageway_Hazards                            2022709
Special_Conditions_at_Site                     2006278
Hit_Object_in_Carriageway                      1973945
Hit_Object_off_Carriageway                     1881426
Skidding_and_Overturning                       1798784
2nd_Road_Class                                  827937
Driver_IMD_Decile                               689290
Age_of_Vehicle                                  337962
model                                           299357
Engine_Capacity_.CC.                            250317
Propulsion_Code                                 233598
LSOA_of_Accident_Location                       139207
make                                            110845
2nd_Road_Number                                  18927
Pedestrian_Crossing-Physical_Facilities           1370
Vehicle_Location.Restricted_Lane                  1124
Pedestrian_Crossing-Human_Control                  654
Time                                               146
Longitude 

In [5]:
train_raw.drop( columns= ['Special_Conditions_at_Site','Carriageway_Hazards','Hit_Object_in_Carriageway','Hit_Object_off_Carriageway','Skidding_and_Overturning','2nd_Road_Class','Driver_IMD_Decile','Year_y'], axis = 1, inplace = True)

In [6]:
train_raw.isnull().sum().sort_values( ascending = False)

Age_of_Vehicle                                 337962
model                                          299357
Engine_Capacity_.CC.                           250317
Propulsion_Code                                233598
LSOA_of_Accident_Location                      139207
make                                           110845
2nd_Road_Number                                 18927
Pedestrian_Crossing-Physical_Facilities          1370
Vehicle_Location.Restricted_Lane                 1124
Pedestrian_Crossing-Human_Control                 654
Time                                              146
Longitude                                         125
Latitude                                          124
Location_Easting_OSGR                             124
Location_Northing_OSGR                            124
Did_Police_Officer_Attend_Scene_of_Accident       114
Speed_limit                                        65
InScotland                                         44
1st_Road_Number             

In [7]:
#Replace null vlaues
#Fill the number columns
number_columns_to_fill = [
    '2nd_Road_Number',
    'Pedestrian_Crossing-Physical_Facilities',
    'Pedestrian_Crossing-Human_Control',
    '1st_Road_Number',
]

for column in number_columns_to_fill:
    train_raw[column].fillna(train_raw[column].mean(), inplace=True)
 
#Fill the categorical columns    
object_columns_to_fill = [
    'Did_Police_Officer_Attend_Scene_of_Accident',
    'InScotland',
    'Vehicle_Location.Restricted_Lane'
]

for column in object_columns_to_fill:
    train_raw[column].fillna(train_raw[column].mode()[0], inplace= True)


In [8]:
#Imputing columns with geographical data
# Initialize KNN imputer
imputer = KNNImputer(n_neighbors = 6)

# List of columns to impute
columns_to_impute = ['Latitude',
                     'Longitude',
                     'Location_Easting_OSGR',
                    'Location_Northing_OSGR'
]

# Fit and transform the data
train_raw[columns_to_impute] = pd.DataFrame(imputer.fit_transform(train_raw[columns_to_impute]), columns=columns_to_impute)
train_raw.head(10)

,Accident_Index,1st_Road_Class,1st_Road_Number,2nd_Road_Number,Accident_Severity,Date,Day_of_Week,Did_Police_Officer_Attend_Scene_of_Accident,Junction_Control,Junction_Detail,...,Propulsion_Code,Sex_of_Driver,Towing_and_Articulation,Vehicle_Leaving_Carriageway,Vehicle_Location.Restricted_Lane,Vehicle_Manoeuvre,Vehicle_Reference,Vehicle_Type,Was_Vehicle_Left_Hand_Drive,X1st_Point_of_Impact
0,200501BS00002,B,450.0,0.0,Slight,2005-01-05,Wednesday,1.0,Auto traffic signal,Crossroads,...,Heavy oil,Male,No tow/articulation,Did not leave carriageway,0.0,Slowing or stopping,1,Bus or coach (17 or more pass seats),No,Nearside
1,200501BS00003,C,0.0,0.0,Slight,2005-01-06,Thursday,1.0,Data missing or out of range,Not at junction or within 20 metres,...,Heavy oil,Male,No tow/articulation,Did not leave carriageway,0.0,Going ahead right-hand bend,1,Bus or coach (17 or more pass seats),No,Nearside
2,200501BS00004,A,3220.0,0.0,Slight,2005-01-07,Friday,1.0,Data missing or out of range,Not at junction or within 20 metres,...,Petrol,Female,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Car,No,Front
3,200501BS00005,Unclassified,0.0,0.0,Slight,2005-01-10,Monday,1.0,Data missing or out of range,Not at junction or within 20 metres,...,Petrol,Male,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Motorcycle 125cc and under,No,Front
4,200501BS00006,Unclassified,0.0,0.0,Slight,2005-01-11,Tuesday,1.0,Data missing or out of range,Not at junction or within 20 metres,...,Petrol,Male,No tow/articulation,Did not leave carriageway,0.0,Moving off,1,Car,No,Did not impact
5,200501BS00006,Unclassified,0.0,0.0,Slight,2005-01-11,Tuesday,1.0,Data missing or out of range,Not at junction or within 20 metres,...,Petrol,Female,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,2,Motorcycle 125cc and under,No,Did not impact
6,200501BS00007,C,0.0,0.0,Slight,2005-01-13,Thursday,1.0,Give way or uncontrolled,T or staggered junction,...,NaN,Male,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Motorcycle 125cc and under,No,Front
7,200501BS00007,C,0.0,0.0,Slight,2005-01-13,Thursday,1.0,Give way or uncontrolled,T or staggered junction,...,Petrol,Male,No tow/articulation,Did not leave carriageway,0.0,Parked,2,Car,No,Back
8,200501BS00009,A,315.0,0.0,Slight,2005-01-14,Friday,1.0,Data missing or out of range,Not at junction or within 20 metres,...,Petrol,Male,No tow/articulation,Nearside,0.0,Going ahead other,1,Car,No,Front
9,200501BS00012,A,4.0,325.0,Slight,2005-01-16,Sunday,1.0,Auto traffic signal,Crossroads,...,Petrol,Male,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Car,No,Front


In [9]:
train_raw.groupby('Road_Type')['Speed_limit'].count()
mode_speed = train_raw.groupby('Road_Type')['Speed_limit'].apply(lambda x: x.mode()[0] if not x.mode().empty else None)
#fill nulls based on the mode
def fill_speed_limit(row):
    if pd.isnull(row['Speed_limit']):
        return mode_speed[row['Road_Type']]
    else:
        return row['Speed_limit']

#fill nulls in 'Speed_limit'
train_raw['Speed_limit'] = train_raw.apply(fill_speed_limit, axis=1)

In [10]:
train_raw.isnull().sum().sort_values( ascending = False)

Age_of_Vehicle                                 337962
model                                          299357
Engine_Capacity_.CC.                           250317
Propulsion_Code                                233598
LSOA_of_Accident_Location                      139207
make                                           110845
Time                                              146
Vehicle_Leaving_Carriageway                         0
Towing_and_Articulation                             0
Urban_or_Rural_Area                                 0
Weather_Conditions                                  0
Year_x                                              0
InScotland                                          0
Age_Band_of_Driver                                  0
Was_Vehicle_Left_Hand_Drive                         0
Vehicle_Type                                        0
Driver_Home_Area_Type                               0
Vehicle_Reference                                   0
Journey_Purpose_of_Driver   

In [11]:
#Handle missing values for datetime variables

#Convert to datetime
train_raw['Time'] = pd.to_datetime(train_raw['Time'], format='%H:%M', errors='coerce')

#Impute the null values of the datetime column
# Step 2: Find the most frequent time
most_frequent_time = train_raw['Time'].mode()[0]

# Step 3: Fill null values with the most frequent time
train_raw['Time'].fillna(most_frequent_time, inplace=True)

In [12]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Convert 'Propulsion_Code' to numeric
label_encoder = LabelEncoder()
train_raw['Propulsion_Code'] = label_encoder.fit_transform(train_raw['Propulsion_Code'])

# Select features to predict missing 'model' values
features = ['Age_of_Vehicle', 'Engine_Capacity_.CC.', 'Propulsion_Code']
numeric_data = train_raw[features]

# Impute missing values in numeric data
knn_imputer = KNNImputer(n_neighbors = 3)
numeric_data_imputed = knn_imputer.fit_transform(numeric_data)

# Replace the original dataframe with the imputed values
train_raw[features] = numeric_data_imputed

In [13]:
train_raw.isnull().sum().sort_values( ascending = False)

model                                          299357
LSOA_of_Accident_Location                      139207
make                                           110845
Junction_Location                                   0
Urban_or_Rural_Area                                 0
Weather_Conditions                                  0
Year_x                                              0
InScotland                                          0
Age_Band_of_Driver                                  0
Age_of_Vehicle                                      0
Driver_Home_Area_Type                               0
Engine_Capacity_.CC.                                0
Journey_Purpose_of_Driver                           0
Accident_Index                                      0
Speed_limit                                         0
Propulsion_Code                                     0
Sex_of_Driver                                       0
Towing_and_Articulation                             0
Vehicle_Leaving_Carriageway 